#Install Server Requirements
This should be run on a minimum of a T4 runtime, though it will run on a CPU only session, however long TTS generations may time out/error.

This is a **work in progress**. Known issues:

- The 1st TTS generation has a brief stutter.
- RVC is not yet working.
- Transcoding/ffmpeg isnt working.
- Things yet to do on selecting your first model and other configuration setups.

If you enable DeepSpeed for XTTS models, DeepSpeed has to compile on the 1st TTS generation which can take about 90 seconds. After that it should be fine.

In [ ]:
#@markdown #####You can enable gdrive that drag and drop samples directly through your google drive via the `drive/Mydrive` path
mount_gdrive = False #@param{type:"boolean"}

if mount_gdrive:
  from google.colab import drive
  drive.mount('/content/drive')
from IPython.display import clear_output
print("*******************************************************************")
print("** Installing server requirements. This will take 5-10 minutes ****")
print("*******************************************************************")
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 libaio-dev espeak-ng > '/dev/null' 2>&1
clear_output()
print("************************")
print("*** Cloning AllTalk ****")
print("************************\n")
!git clone -b alltalkbeta https://github.com/erew123/alltalk_tts
clear_output()
print("\n********************************")
print("*** Installing Requirements ****")
print("********************************\n")
!python -m pip install --upgrade "pip<24.1"
!pip install --quiet -r /content/alltalk_tts/system/requirements/requirements_colab.txt
clear_output()
print("\n*****************************")
print("*** Installing DeepSpeed ****")
print("*****************************\n")
!pip install deepspeed
!pip install orjson
!pip install faiss-cpu
!pip install fairseq
clear_output()
print("\n******************************")
print("*** Installing Cloudflare ****")
print("******************************\n")
# Install cloudflare
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb > '/dev/null' 2>&1
!apt install ./cloudflared-linux-amd64.deb aria2 > '/dev/null' 2>&1
!rm cloudflared-linux-amd64.deb > '/dev/null' 2>&1
!python -m spacy download en_core_web_md
clear_output()
print("\n***************************")
print("*** Installing Cutlass ****")
print("***************************\n")
# Clone the CUTLASS repository
!git clone https://github.com/NVIDIA/cutlass.git
!export CUTLASSPATH=/content/cutlass
!sudo curl -L https://github.com/BtbN/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz -o /usr/local/bin/ffmpeg.tar.xz
clear_output()
%cd /usr/local/bin/
clear_output()
!7z e /usr/local/bin/ffmpeg.tar.xz -y
clear_output()
!7z e /usr/local/bin/ffmpeg.tar -y
clear_output()
!sudo chmod a+rx /usr/local/bin/ffmpeg
clear_output()
!pip uninstall jax -y
clear_output()
print("************************************")
print("** Server requirements installed ***")
print("*** Please proceed to next step ****")
print("************************************\n")


********************************
*** Installing Requirements ****
********************************

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
 


# Start AllTalk

In [ ]:
#@markdown Links to the API and Gradio interface will be shown when AllTalk has loaded in.<br>
#@markdown The API address is what you would use in SillyTavern.
import re
import time
import json
import threading
import time

def keep_alive():
    while True:
        time.sleep(60)  # Run every 60 seconds (adjust as needed)

keep_alive_thread = threading.Thread(target=keep_alive)
keep_alive_thread.start()

Tunnel = "cloudflare"
host = "127.0.0.1"
ports = [7851, 7852]
tunnel_urls = []

# Starting tunnels for each port.
for port in ports:
    if Tunnel == "cloudflare":
        !nohup cloudflared tunnel --url http://{host}:{port} > lt_{port}.log 2>&1 &
    else:
        !nohup npx lt -p {port} > lt_{port}.log 2>&1 &

# Wait a couple of seconds for the tunnels to initialize.
time.sleep(10)

# Extract URLs for each tunnel.
for port in ports:
    log_file = f'lt_{port}.log'
    with open(log_file, 'r') as testwritefile:
        log_content = testwritefile.read()

        # Use regular expressions to find the URL.
        if Tunnel == "cloudflare":
            url_match = re.search(r'(https://[-a-z0-9]+\.trycloudflare\.com)', log_content)
        else:
            url_match = re.search(r'your url is: (https?://\S+)', log_content)

        if url_match:
            tunnel_url = url_match.group(1)
            tunnel_urls.append(tunnel_url)
        else:
            print(f"URL for port {port} not found.")

# Save the tunnel URLs to a JSON file.
try:
    # Try to open the JSON file for reading.
    with open('/content/alltalk_tts/googlecolab.json', 'r') as f:
        data = json.load(f)
except FileNotFoundError:
    # If the file doesn't exist, create an empty dictionary.
    data = {}

data['google_ip_address'] = tunnel_urls

# Write the modified data (or newly created data) back to the file.
with open('/content/alltalk_tts/googlecolab.json', 'w') as f:
    json.dump(data, f)

host = "0.0.0.0"

if Tunnel == "localtunnel":
    print("Before you copy the link above click on it and copy that ip:")
    !curl ipv4.icanhazip.com

# Start API server.
!python /content/alltalk_tts/script.py
